In [0]:
# Cell 0: Imports
import os
import json
import base64
import uuid
import pandas as pd
from io import BytesIO
from datetime import datetime
from pyspark.sql import functions as F
from pyspark.sql.functions import lit, current_timestamp
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, ArrayType
from delta.tables import DeltaTable

from clinical_data_standards_framework.utils import (
    save_with_audit,
    get_codelist_flexible_mapper,
    find_excel_header_row,
    get_header_filter_patterns
)


In [ ]:
# Cell 1: Check Upstream Status & Documents Found
# 1. Check if transfer variables processing succeeded
# 2. Check if extract_excel_sheets found documents

# Check upstream task status (transfer variables)
print("Checking upstream task status...")
try:
    upstream_status = dbutils.jobs.taskValues.get(
        taskKey="load_transfer_metadata", 
        key="transfer_variables_status"
    )
    print(f"  Upstream status (load_transfer_metadata): {upstream_status}")
    
    if upstream_status == "FAILED":
        print("\n⚠️  Upstream task (load_transfer_metadata) FAILED")
        print("   Codelist processing requires successful transfer variable processing.")
        print("   Exiting...")
        dbutils.notebook.exit("SKIPPED - Upstream task failed")
    elif upstream_status == "UP_TO_DATE":
        print("  Upstream reports all sheets already processed")
    elif upstream_status == "SUCCESS":
        success_count = dbutils.jobs.taskValues.get(taskKey="load_transfer_metadata", key="transfer_variables_success_count")
        error_count = dbutils.jobs.taskValues.get(taskKey="load_transfer_metadata", key="transfer_variables_error_count")
        print(f"  Upstream processed: {success_count} success, {error_count} errors")
    
except Exception as e:
    print(f"⚠ Could not read upstream status: {e}")
    print("  Continuing anyway (backwards compatibility)")

# Check if documents were found
try:
    documents_found = dbutils.jobs.taskValues.get(taskKey="extract_excel_sheets", key="documents_found")
    
    if documents_found == "false":
        print("\n⏭️  No documents were found in previous task (extract_excel_sheets)")
        print("   Skipping codelist processing.")
        dbutils.notebook.exit("SKIPPED - no documents to process")
    
    print(f"✓ Documents found in previous task: {documents_found}")
    
except Exception as e:
    print(f"⚠ Could not read 'documents_found' task value: {e}")
    print("  Continuing anyway (backwards compatibility)")
    pass

print("\n✓ All upstream checks passed. Proceeding with codelist processing...")

In [0]:
# Cell 2: Get Configuration from Setup Task
globals_dict = json.loads(dbutils.jobs.taskValues.get(taskKey="setup", key="globals"))
pipeline_config = json.loads(dbutils.jobs.taskValues.get(taskKey="setup", key="pipeline_config"))
created_by_principal = dbutils.jobs.taskValues.get(taskKey="setup", key="created_by_principal")
databricks_job_id = dbutils.jobs.taskValues.get(taskKey="setup", key="databricks_job_id")
databricks_job_name = dbutils.jobs.taskValues.get(taskKey="setup", key="databricks_job_name")
databricks_run_id = dbutils.jobs.taskValues.get(taskKey="setup", key="databricks_run_id")

# Extract needed values from globals
catalog = globals_dict['catalog']
bronze_schema = globals_dict['bronze_schema']
silver_schema = globals_dict['silver_schema']

print(f"Catalog: {catalog}")
print(f"Bronze Schema: {bronze_schema}")
print(f"Silver Schema: {silver_schema}")
print(f"Created by: {created_by_principal}")
print(f"Job ID: {databricks_job_id}, Job Name: {databricks_job_name}, Run ID: {databricks_run_id}")

In [0]:
# Cell 3: Construct Table Names
sheets_table = f"{catalog}.{bronze_schema}.md_dta_excel_file_raw"
history_table = f"{catalog}.{bronze_schema}.md_dta_history"
status_table = f"{catalog}.{bronze_schema}.md_document_status"
output_table = f"{catalog}.{silver_schema}.md_codelists_normalized"

print(f"Sheets table: {sheets_table}")
print(f"History table: {history_table}")
print(f"Status table: {status_table}")
print(f"Output table: {output_table}")


In [0]:
# Cell 4: Read Codelist Sheets
# Filter for sheets categorized as 'codelists'

print(f"{'='*80}")
print(f"Reading codelist sheets...")
print(f"{'='*80}")

df_sheets = spark.table(sheets_table).filter(
    F.col("sheet_category") == "codelists"
).select(
    "document_id",
    "parent_document_id",
    "sheet_name",
    "source_file_path",
    "trial_id",
    "data_stream_type",
    "data_provider_name"
)

# Join with parent documents to get content_base64
df_with_content = df_sheets.join(
    spark.table(history_table).select(
        F.col("document_id").alias("parent_id"),
        "content_base64"
    ),
    df_sheets.parent_document_id == F.col("parent_id")
)

sheet_count = df_with_content.count()
print(f"Found {sheet_count} codelist sheets to process")
print(f"{'='*80}")

if sheet_count == 0:
    print("No codelist sheets to process. Exiting.")
    dbutils.notebook.exit("No codelist sheets to process")

In [0]:
# Cell 5: Process Codelist Sheets
# Parse codelist sheets and aggregate values with SDTM mappings

print(f"{'='*80}")
print(f"Processing codelist sheets...")
print(f"{'='*80}")

# Dictionary: key = (trial, stream, provider, var_name), value = list of {value, sdtm_value}
all_codelists = {}

for row in df_with_content.collect():
    sheet_name = row['sheet_name']
    source_path = row['source_file_path']
    content_b64 = row['content_base64']
    
    trial_id = row['trial_id']
    data_stream_type = row['data_stream_type']
    data_provider_name = row['data_provider_name']
    
    print(f"\nProcessing: {sheet_name}")
    print(f"  Trial: {trial_id}, Stream: {data_stream_type}, Provider: {data_provider_name}")
    
    try:
        # Decode Excel from base64
        excel_bytes = base64.b64decode(content_b64)
        file_ext = os.path.splitext(source_path.lower())[1]
        engine = 'xlrd' if file_ext in ['.xls', '.xlt'] else 'openpyxl'
        
        # Read sheet to find header row
        raw_df = pd.read_excel(BytesIO(excel_bytes), sheet_name=sheet_name, header=None, engine=engine)
        header_row = find_excel_header_row(raw_df, [
            r'codelist|reference',
            r'values?|code'
        ])
        
        # Read sheet with proper header
        df_sheet = pd.read_excel(
            BytesIO(excel_bytes),
            sheet_name=sheet_name,
            header=header_row,
            engine=engine
        ).dropna(axis=1, how='all')
        
        print(f"  Original columns: {list(df_sheet.columns)[:5]}...")
        print(f"  Total rows before any filtering: {len(df_sheet)}")
        
        # Use flexible mapper
        col_map, missing_required = get_codelist_flexible_mapper(list(df_sheet.columns))
        
        print(f"  Mapped {len(col_map)} columns")
        has_sdtm = 'sdtm_value' in col_map.values()
        print(f"  SDTM mapping column: {'Found' if has_sdtm else 'Not found'}")
        
        # Check for missing required columns
        if missing_required:
            print(f"  ⚠ WARNING: Missing required columns: {', '.join(missing_required)}")
            print(f"  Skipping sheet")
            continue
        
        # Rename columns
        df_sheet = df_sheet.rename(columns=col_map)
        
        # Filter rows with both reference and value
        df_sheet = df_sheet[
            df_sheet['codelist_reference'].notna() & 
            df_sheet['code_value'].notna()
        ].copy()
        
        print(f"  Found {len(df_sheet)} codelist value entries")
        
        # Group by codelist_reference and collect values with mappings
        for var_name, grp in df_sheet.groupby('codelist_reference'):
            var_name_clean = str(var_name).strip()
            if not var_name_clean:
                continue
            
            key = (trial_id, data_stream_type, data_provider_name, var_name_clean)
            
            if key not in all_codelists:
                all_codelists[key] = []
            
            # Collect value-sdtm pairs
            for _, r in grp.iterrows():
                value = str(r.get('code_value', '')).strip()
                if value and value.lower() != 'nan':
                    sdtm = None
                    if has_sdtm and 'sdtm_value' in grp.columns:
                        sdtm_raw = r.get('sdtm_value')
                        if pd.notna(sdtm_raw):
                            sdtm = str(sdtm_raw).strip()
                            if sdtm.lower() == 'nan' or sdtm == '':
                                sdtm = None
                    
                    # Store as dict for later conversion
                    all_codelists[key].append({
                        'value': value,
                        'sdtm_value': sdtm
                    })
        
        unique_codelists = len(df_sheet['codelist_reference'].unique())
        print(f"  ✓ Processed {unique_codelists} unique codelist references")
        
    except Exception as e:
        print(f"  ✗ ERROR: {str(e)}")
        import traceback
        traceback.print_exc()
        continue

# Convert to records for DataFrame
records = []
for (trial_id, data_stream_type, data_provider_name, var_name), mappings_list in all_codelists.items():
    # Extract unique values (for simple array)
    values_set = set()
    value_sdtm_mappings = []
    
    for mapping in mappings_list:
        val = mapping['value']
        values_set.add(val)
        
        # Add to mapping array (include even if sdtm_value is None)
        value_sdtm_mappings.append({
            'value': val,
            'sdtm_value': mapping['sdtm_value']
        })
    
    # Sort for consistency
    values_list = sorted(list(values_set))
    
    # Deduplicate mappings (in case same value appears multiple times)
    unique_mappings = {}
    for m in value_sdtm_mappings:
        if m['value'] not in unique_mappings:
            unique_mappings[m['value']] = m
    
    value_sdtm_list = sorted(list(unique_mappings.values()), key=lambda x: x['value'])
    
    records.append({
        'codelist_id': str(uuid.uuid4()),
        'trial_id': trial_id,
        'data_stream_type': data_stream_type,
        'data_provider_name': data_provider_name,
        'transfer_variable_name': var_name,
        'values': values_list,
        'value_sdtm_mapping': value_sdtm_list,
        'row_status': None,
        'categorization_notes': None
    })

print(f"\n{'='*80}")
print(f"✓ Processing complete: {len(records)} unique codelists")
print(f"{'='*80}")

if len(records) == 0:
    print("No codelists to write. Exiting.")
    dbutils.notebook.exit("No codelists found to process")


In [0]:
# Cell 6: Write Results to Silver Tables & Set Task Values
# Create DataFrames, write with audit columns, and signal success/failure

if len(all_codelist_records) == 0:
    print("No codelist records to write. Exiting.")
    dbutils.jobs.taskValues.set(key="codelist_status", value="NO_DATA")
    dbutils.jobs.taskValues.set(key="codelist_success_count", value="0")
    dbutils.jobs.taskValues.set(key="codelist_error_count", value="0")
    dbutils.notebook.exit("No codelists found to process")

# Count success vs error records
success_count = len([r for r in all_codelist_records if r.get('row_status') == 'COMPLETED'])
error_count = len([r for r in all_codelist_records if r.get('row_status') and r['row_status'] != 'COMPLETED'])

print(f"\n{'='*80}")
print(f"Processing Summary:")
print(f"  Total records: {len(all_codelist_records)}")
print(f"  Success (COMPLETED): {success_count}")
print(f"  Errors/Manual Review: {error_count}")
print(f"{'='*80}")

# Create Spark DataFrame for parent codelist table
df_codelists = spark.createDataFrame(all_codelist_records, schema=codelist_output_schema)

# Display sample
print(f"\nSample codelist records:")
df_codelists.show(5, truncate=False)

# Write to silver table with audit columns using MERGE mode
print(f"\nWriting {len(all_codelist_records)} records to {output_parent_table}...")
save_with_audit(
    df=df_codelists,
    table_name=output_parent_table,
    created_by_principal=created_by_principal,
    databricks_job_id=databricks_job_id,
    databricks_job_name=databricks_job_name,
    databricks_run_id=databricks_run_id,
    mode='merge',  # Use merge for incremental updates
    merge_keys=['trial_id', 'data_stream_type', 'data_provider_name', 'codelist_id']
)

print(f"\n{'='*80}")
print(f"✓ Successfully wrote {len(all_codelist_records)} codelist records")
print(f"  Table: {output_parent_table}")
print(f"  Mode: merge (incremental updates)")
print(f"  Merge keys: trial_id, data_stream_type, data_provider_name, codelist_id")
print(f"{'='*80}")

# Set task values for monitoring
if success_count == 0 and error_count > 0:
    task_status = "FAILED"
    dbutils.jobs.taskValues.set(key="codelist_status", value=task_status)
    dbutils.jobs.taskValues.set(key="codelist_success_count", value=str(success_count))
    dbutils.jobs.taskValues.set(key="codelist_error_count", value=str(error_count))
    raise Exception(f"All {error_count} codelist records failed. Check categorization_notes for details.")
elif success_count > 0:
    task_status = "SUCCESS"
    dbutils.jobs.taskValues.set(key="codelist_status", value=task_status)
    dbutils.jobs.taskValues.set(key="codelist_success_count", value=str(success_count))
    dbutils.jobs.taskValues.set(key="codelist_error_count", value=str(error_count))
    print(f"\n✓ Task status: {task_status}")
else:
    task_status = "NO_DATA"
    dbutils.jobs.taskValues.set(key="codelist_status", value=task_status)
    dbutils.jobs.taskValues.set(key="codelist_success_count", value="0")
    dbutils.jobs.taskValues.set(key="codelist_error_count", value="0")

In [0]:
# Cell 7: Verify Results

print(f"\nVerifying table: {output_table}")
result_count = spark.table(output_table).count()
print(f"Total codelists: {result_count}")

print(f"\nSample codelists:")
spark.table(output_table).select(
    "codelist_id",
    "trial_id",
    "transfer_variable_name",
    "values",
    "value_sdtm_mapping",
    "databricks_job_name"
).show(5, truncate=False)

# Show exploded mappings for verification
print(f"\nExample: Exploded value->SDTM mappings:")
spark.table(output_table).select(
    "transfer_variable_name",
    F.explode("value_sdtm_mapping").alias("mapping")
).select(
    "transfer_variable_name",
    "mapping.value",
    "mapping.sdtm_value"
).limit(10).show(truncate=False)
